# Delete duplicate values from POI

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import sys
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/edgaregurrola/Repos/observatorio-ciudades/accesibilidad-urbana/aup/data.py:24: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


## Define required treatment

In [11]:
source_list = ['','','',
                   'supermercado','clinica_priv','clinica_pub',
                   'hospital_priv','hospital_pub','',
                   'consult_ado_priv','consult_ado_pub',
                   '','eq_deportivo_pub','',
                   'tax_collection','feria','ep_plaza_big',
                   '','museos_priv','museos_pub','sitios_historicos',
                   '','','','edu_basica_priv',
                   'edu_basica_pub','edu_media_priv','edu_media_pub',
                   '','jardin_inf_pub','edu_especial_priv',
                   'edu_especial_pub','bibliotecas']
treat_dict = {('carniceria','supermercado','club_deportivo'):['rut','rzn_scl','geometry'],
           ('hogar','local_mini_market','farmacia',
           'banco','cines','restaurantes_bar_cafe','librerias',
           'jardin_inf_priv'):['rut','empresa','geometry'],
             ('eq_deportivo_priv','eq_deportivo_pub'):['nombre','sector','geometry']}

## Data processing

In [13]:
grl_dir = '../../../data/processed/00_pois_formated/'

In [19]:
for key in treat_dict.keys():
    for source in key:
        gdf = gpd.read_file(grl_dir+source+'.gpkg')
        gdf.columns= gdf.columns.str.lower()
        aup.log(f'Current data for {source} is {len(gdf)}')
        gdf = gdf.drop_duplicates(treat_dict[key])
        aup.log(f'After processing data for {source} is reduced to {len(gdf)}')
        gdf.to_file(grl_dir+source+'.gpkg')